In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
from __future__ import print_function
from util_functions import process_files

import keras
from keras.models import Sequential, load_model, model_from_json
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

import numpy as np
from numpy import linalg as LA

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/matplotlib/__init__.py:962: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #2
  (fname, cnt))
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/matplotlib/__init__.py:962: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #3
  (fname, cnt))
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
batch_size = 32
num_classes = 2
epochs = 1500

# load the data
x_train, y_train, f_train = process_files(dataset='training', features=['mfccs', 'mfcc_deltas'], shape='stacked')
x_test, y_test, f_test = process_files(dataset='test', features=['mfccs', 'mfcc_deltas'], shape='stacked')

#normalise
x_train = x_train - x_train.mean()
x_test = x_test - x_test.mean()
x_train = x_train/LA.norm(x_train)
x_test = x_test/LA.norm(x_test)

# input image dimensions
input_d = x_train.shape[1] #Depth
input_h = x_train.shape[2] #Height
input_w = x_train.shape[3] #Width

#Reshaping to feed to network
x_train = x_train.reshape(x_train.shape[0], input_h*input_w*input_d)
x_test = x_test.reshape(x_test.shape[0], input_h*input_w*input_d)
y_train = y_train.reshape(x_train.shape[0], 1)
y_test = y_test.reshape(x_test.shape[0], 1)

input_dim = input_h*input_w*input_d

#Making them floats for TF
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes=num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes=num_classes)

#Network parameters
learning_rate = 0.0001
dropout_rate = 0.2

model = Sequential()
model.add(Dense(64, activation='relu', input_shape=(input_dim,)))
model.add(Dropout(dropout_rate))
model.add(Dense(128, activation='relu'))
model.add(Dropout(dropout_rate))
model.add(Dense(256, activation='relu'))
model.add(Dropout(dropout_rate))
model.add(Dense(2, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Nadam(lr=learning_rate),
              metrics=['accuracy'])

tbCallBack = keras.callbacks.TensorBoard(log_dir='./Graph', histogram_freq=0,  
      write_graph=True, write_images=True)

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=2,
          callbacks=[tbCallBack])

score = model.evaluate(x_test, y_test, verbose=0)

# serialize model to JSON
model_json = model.to_json()
with open('saved_models/sound_classifier.json', 'w') as json_file:
    json_file.write(model_json)
# serialize weights to HDF5        
model.save_weights('saved_models/sound_classifier.h5')

print('Test loss:', score[0])
print('Test accuracy:', score[1])        

print('----------------------------------------------------\n')    

Epoch 1/1500
 - 1s - loss: 0.6836 - acc: 0.6956
Epoch 2/1500
 - 0s - loss: 0.6522 - acc: 0.6963
Epoch 3/1500
 - 0s - loss: 0.6202 - acc: 0.6963
Epoch 4/1500
 - 0s - loss: 0.6090 - acc: 0.6963
Epoch 5/1500
 - 0s - loss: 0.6047 - acc: 0.6963
Epoch 6/1500
 - 0s - loss: 0.5986 - acc: 0.6963
Epoch 7/1500
 - 0s - loss: 0.5916 - acc: 0.6963
Epoch 8/1500
 - 0s - loss: 0.5769 - acc: 0.6963
Epoch 9/1500
 - 0s - loss: 0.5600 - acc: 0.6963
Epoch 10/1500
 - 0s - loss: 0.5344 - acc: 0.6983
Epoch 11/1500
 - 0s - loss: 0.5063 - acc: 0.7521
Epoch 12/1500
 - 0s - loss: 0.4771 - acc: 0.7975
Epoch 13/1500
 - 0s - loss: 0.4518 - acc: 0.8237
Epoch 14/1500
 - 0s - loss: 0.4288 - acc: 0.8292
Epoch 15/1500
 - 0s - loss: 0.4144 - acc: 0.8437
Epoch 16/1500
 - 0s - loss: 0.4004 - acc: 0.8423
Epoch 17/1500
 - 0s - loss: 0.3891 - acc: 0.8554
Epoch 18/1500
 - 0s - loss: 0.3795 - acc: 0.8540
Epoch 19/1500
 - 0s - loss: 0.3654 - acc: 0.8616
Epoch 20/1500
 - 0s - loss: 0.3630 - acc: 0.8678
Epoch 21/1500
 - 0s - loss: 0

 - 0s - loss: 0.1326 - acc: 0.9504
Epoch 168/1500
 - 0s - loss: 0.1370 - acc: 0.9497
Epoch 169/1500
 - 0s - loss: 0.1340 - acc: 0.9525
Epoch 170/1500
 - 0s - loss: 0.1322 - acc: 0.9552
Epoch 171/1500
 - 0s - loss: 0.1344 - acc: 0.9477
Epoch 172/1500
 - 0s - loss: 0.1448 - acc: 0.9435
Epoch 173/1500
 - 0s - loss: 0.1321 - acc: 0.9539
Epoch 174/1500
 - 0s - loss: 0.1290 - acc: 0.9539
Epoch 175/1500
 - 0s - loss: 0.1367 - acc: 0.9470
Epoch 176/1500
 - 0s - loss: 0.1268 - acc: 0.9573
Epoch 177/1500
 - 0s - loss: 0.1313 - acc: 0.9559
Epoch 178/1500
 - 0s - loss: 0.1276 - acc: 0.9552
Epoch 179/1500
 - 0s - loss: 0.1269 - acc: 0.9573
Epoch 180/1500
 - 0s - loss: 0.1322 - acc: 0.9511
Epoch 181/1500
 - 0s - loss: 0.1204 - acc: 0.9580
Epoch 182/1500
 - 0s - loss: 0.1270 - acc: 0.9518
Epoch 183/1500
 - 0s - loss: 0.1230 - acc: 0.9559
Epoch 184/1500
 - 0s - loss: 0.1232 - acc: 0.9573
Epoch 185/1500
 - 0s - loss: 0.1195 - acc: 0.9594
Epoch 186/1500
 - 0s - loss: 0.1238 - acc: 0.9566
Epoch 187/1500


 - 0s - loss: 0.0551 - acc: 0.9828
Epoch 332/1500
 - 0s - loss: 0.0534 - acc: 0.9828
Epoch 333/1500
 - 0s - loss: 0.0550 - acc: 0.9821
Epoch 334/1500
 - 0s - loss: 0.0588 - acc: 0.9821
Epoch 335/1500
 - 0s - loss: 0.0493 - acc: 0.9835
Epoch 336/1500
 - 0s - loss: 0.0471 - acc: 0.9883
Epoch 337/1500
 - 0s - loss: 0.0541 - acc: 0.9807
Epoch 338/1500
 - 0s - loss: 0.0540 - acc: 0.9835
Epoch 339/1500
 - 0s - loss: 0.0647 - acc: 0.9800
Epoch 340/1500
 - 0s - loss: 0.0508 - acc: 0.9800
Epoch 341/1500
 - 0s - loss: 0.0595 - acc: 0.9842
Epoch 342/1500
 - 0s - loss: 0.0483 - acc: 0.9862
Epoch 343/1500
 - 0s - loss: 0.0509 - acc: 0.9848
Epoch 344/1500
 - 0s - loss: 0.0493 - acc: 0.9855
Epoch 345/1500
 - 0s - loss: 0.0481 - acc: 0.9835
Epoch 346/1500
 - 0s - loss: 0.0476 - acc: 0.9855
Epoch 347/1500
 - 0s - loss: 0.0497 - acc: 0.9855
Epoch 348/1500
 - 0s - loss: 0.0502 - acc: 0.9835
Epoch 349/1500
 - 0s - loss: 0.0491 - acc: 0.9890
Epoch 350/1500
 - 0s - loss: 0.0504 - acc: 0.9855
Epoch 351/1500


 - 0s - loss: 0.0243 - acc: 0.9938
Epoch 496/1500
 - 0s - loss: 0.0223 - acc: 0.9917
Epoch 497/1500
 - 0s - loss: 0.0251 - acc: 0.9938
Epoch 498/1500
 - 0s - loss: 0.0219 - acc: 0.9945
Epoch 499/1500
 - 0s - loss: 0.0240 - acc: 0.9945
Epoch 500/1500
 - 0s - loss: 0.0258 - acc: 0.9931
Epoch 501/1500
 - 0s - loss: 0.0265 - acc: 0.9924
Epoch 502/1500
 - 0s - loss: 0.0267 - acc: 0.9945
Epoch 503/1500
 - 0s - loss: 0.0278 - acc: 0.9904
Epoch 504/1500
 - 0s - loss: 0.0280 - acc: 0.9917
Epoch 505/1500
 - 0s - loss: 0.0250 - acc: 0.9924
Epoch 506/1500
 - 0s - loss: 0.0238 - acc: 0.9945
Epoch 507/1500
 - 0s - loss: 0.0244 - acc: 0.9931
Epoch 508/1500
 - 0s - loss: 0.0221 - acc: 0.9945
Epoch 509/1500
 - 0s - loss: 0.0276 - acc: 0.9924
Epoch 510/1500
 - 0s - loss: 0.0213 - acc: 0.9945
Epoch 511/1500
 - 0s - loss: 0.0214 - acc: 0.9952
Epoch 512/1500
 - 0s - loss: 0.0226 - acc: 0.9924
Epoch 513/1500
 - 0s - loss: 0.0235 - acc: 0.9910
Epoch 514/1500
 - 0s - loss: 0.0218 - acc: 0.9959
Epoch 515/1500


 - 0s - loss: 0.0071 - acc: 0.9986
Epoch 660/1500
 - 0s - loss: 0.0101 - acc: 0.9966
Epoch 661/1500
 - 0s - loss: 0.0089 - acc: 0.9979
Epoch 662/1500
 - 0s - loss: 0.0120 - acc: 0.9966
Epoch 663/1500
 - 0s - loss: 0.0114 - acc: 0.9966
Epoch 664/1500
 - 0s - loss: 0.0208 - acc: 0.9938
Epoch 665/1500
 - 0s - loss: 0.0100 - acc: 0.9959
Epoch 666/1500
 - 0s - loss: 0.0060 - acc: 0.9986
Epoch 667/1500
 - 0s - loss: 0.0055 - acc: 1.0000
Epoch 668/1500
 - 0s - loss: 0.0040 - acc: 0.9993
Epoch 669/1500
 - 0s - loss: 0.0158 - acc: 0.9959
Epoch 670/1500
 - 0s - loss: 0.0069 - acc: 0.9966
Epoch 671/1500
 - 0s - loss: 0.0101 - acc: 0.9959
Epoch 672/1500
 - 0s - loss: 0.0106 - acc: 0.9952
Epoch 673/1500
 - 0s - loss: 0.0174 - acc: 0.9931
Epoch 674/1500
 - 0s - loss: 0.0062 - acc: 0.9972
Epoch 675/1500
 - 0s - loss: 0.0094 - acc: 0.9979
Epoch 676/1500
 - 0s - loss: 0.0158 - acc: 0.9945
Epoch 677/1500
 - 0s - loss: 0.0105 - acc: 0.9972
Epoch 678/1500
 - 0s - loss: 0.0089 - acc: 0.9966
Epoch 679/1500


 - 0s - loss: 0.0029 - acc: 0.9993
Epoch 824/1500
 - 0s - loss: 0.0061 - acc: 0.9972
Epoch 825/1500
 - 0s - loss: 0.0070 - acc: 0.9972
Epoch 826/1500
 - 0s - loss: 0.0101 - acc: 0.9972
Epoch 827/1500
 - 0s - loss: 0.0025 - acc: 1.0000
Epoch 828/1500
 - 0s - loss: 0.0039 - acc: 0.9993
Epoch 829/1500
 - 0s - loss: 0.0040 - acc: 0.9979
Epoch 830/1500
 - 0s - loss: 0.0017 - acc: 1.0000
Epoch 831/1500
 - 0s - loss: 0.0057 - acc: 0.9979
Epoch 832/1500
 - 0s - loss: 0.0097 - acc: 0.9972
Epoch 833/1500
 - 0s - loss: 0.0066 - acc: 0.9972
Epoch 834/1500
 - 0s - loss: 0.0070 - acc: 0.9979
Epoch 835/1500
 - 0s - loss: 0.0064 - acc: 0.9966
Epoch 836/1500
 - 0s - loss: 0.0064 - acc: 0.9966
Epoch 837/1500
 - 0s - loss: 0.0097 - acc: 0.9979
Epoch 838/1500
 - 0s - loss: 0.0051 - acc: 0.9986
Epoch 839/1500
 - 0s - loss: 0.0110 - acc: 0.9952
Epoch 840/1500
 - 0s - loss: 0.0076 - acc: 0.9972
Epoch 841/1500
 - 0s - loss: 0.0045 - acc: 0.9986
Epoch 842/1500
 - 0s - loss: 0.0068 - acc: 0.9972
Epoch 843/1500


 - 0s - loss: 0.0065 - acc: 0.9979
Epoch 988/1500
 - 0s - loss: 0.0133 - acc: 0.9945
Epoch 989/1500
 - 0s - loss: 0.0090 - acc: 0.9979
Epoch 990/1500
 - 0s - loss: 0.0302 - acc: 0.9897
Epoch 991/1500
 - 0s - loss: 0.0156 - acc: 0.9945
Epoch 992/1500
 - 0s - loss: 0.0068 - acc: 0.9966
Epoch 993/1500
 - 0s - loss: 0.0030 - acc: 0.9993
Epoch 994/1500
 - 0s - loss: 0.0048 - acc: 0.9986
Epoch 995/1500
 - 0s - loss: 0.0037 - acc: 0.9993
Epoch 996/1500
 - 0s - loss: 0.0051 - acc: 0.9986
Epoch 997/1500
 - 0s - loss: 0.0039 - acc: 0.9993
Epoch 998/1500
 - 0s - loss: 0.0072 - acc: 0.9979
Epoch 999/1500
 - 0s - loss: 0.0070 - acc: 0.9979
Epoch 1000/1500
 - 0s - loss: 0.0041 - acc: 0.9979
Epoch 1001/1500
 - 0s - loss: 0.0023 - acc: 1.0000
Epoch 1002/1500
 - 0s - loss: 0.0054 - acc: 0.9972
Epoch 1003/1500
 - 0s - loss: 0.0042 - acc: 0.9986
Epoch 1004/1500
 - 0s - loss: 0.0039 - acc: 0.9986
Epoch 1005/1500
 - 0s - loss: 0.0072 - acc: 0.9979
Epoch 1006/1500
 - 0s - loss: 0.0113 - acc: 0.9966
Epoch 10

Epoch 1148/1500
 - 0s - loss: 0.0121 - acc: 0.9924
Epoch 1149/1500
 - 0s - loss: 0.0114 - acc: 0.9959
Epoch 1150/1500
 - 0s - loss: 0.0078 - acc: 0.9979
Epoch 1151/1500
 - 0s - loss: 0.0054 - acc: 0.9979
Epoch 1152/1500
 - 0s - loss: 9.0189e-04 - acc: 1.0000
Epoch 1153/1500
 - 0s - loss: 0.0018 - acc: 0.9993
Epoch 1154/1500
 - 0s - loss: 0.0015 - acc: 1.0000
Epoch 1155/1500
 - 0s - loss: 0.0012 - acc: 1.0000
Epoch 1156/1500
 - 0s - loss: 0.0073 - acc: 0.9979
Epoch 1157/1500
 - 0s - loss: 0.0020 - acc: 0.9993
Epoch 1158/1500
 - 0s - loss: 0.0015 - acc: 1.0000
Epoch 1159/1500
 - 0s - loss: 0.0050 - acc: 0.9986
Epoch 1160/1500
 - 0s - loss: 0.0040 - acc: 0.9986
Epoch 1161/1500
 - 0s - loss: 0.0042 - acc: 0.9979
Epoch 1162/1500
 - 0s - loss: 0.0023 - acc: 0.9993
Epoch 1163/1500
 - 0s - loss: 0.0048 - acc: 0.9993
Epoch 1164/1500
 - 0s - loss: 0.0054 - acc: 0.9993
Epoch 1165/1500
 - 0s - loss: 0.0050 - acc: 0.9972
Epoch 1166/1500
 - 0s - loss: 0.0076 - acc: 0.9972
Epoch 1167/1500
 - 0s - los

 - 0s - loss: 0.0018 - acc: 0.9993
Epoch 1307/1500
 - 0s - loss: 0.0030 - acc: 0.9986
Epoch 1308/1500
 - 0s - loss: 0.0024 - acc: 0.9993
Epoch 1309/1500
 - 0s - loss: 0.0019 - acc: 0.9993
Epoch 1310/1500
 - 0s - loss: 9.3689e-04 - acc: 0.9993
Epoch 1311/1500
 - 0s - loss: 0.0011 - acc: 1.0000
Epoch 1312/1500
 - 0s - loss: 9.5156e-04 - acc: 1.0000
Epoch 1313/1500
 - 0s - loss: 2.8596e-04 - acc: 1.0000
Epoch 1314/1500
 - 0s - loss: 5.5947e-04 - acc: 1.0000
Epoch 1315/1500
 - 0s - loss: 2.8496e-04 - acc: 1.0000
Epoch 1316/1500
 - 0s - loss: 0.0109 - acc: 0.9966
Epoch 1317/1500
 - 0s - loss: 0.0275 - acc: 0.9910
Epoch 1318/1500
 - 0s - loss: 0.0119 - acc: 0.9952
Epoch 1319/1500
 - 0s - loss: 0.0017 - acc: 1.0000
Epoch 1320/1500
 - 0s - loss: 0.0019 - acc: 0.9993
Epoch 1321/1500
 - 0s - loss: 0.0076 - acc: 0.9986
Epoch 1322/1500
 - 0s - loss: 0.0091 - acc: 0.9986
Epoch 1323/1500
 - 0s - loss: 0.0015 - acc: 1.0000
Epoch 1324/1500
 - 0s - loss: 9.1353e-04 - acc: 1.0000
Epoch 1325/1500
 - 0s -

Epoch 1463/1500
 - 0s - loss: 4.3160e-04 - acc: 1.0000
Epoch 1464/1500
 - 0s - loss: 0.0068 - acc: 0.9972
Epoch 1465/1500
 - 0s - loss: 0.0279 - acc: 0.9924
Epoch 1466/1500
 - 0s - loss: 0.0054 - acc: 0.9979
Epoch 1467/1500
 - 0s - loss: 0.0055 - acc: 0.9986
Epoch 1468/1500
 - 0s - loss: 0.0055 - acc: 0.9979
Epoch 1469/1500
 - 0s - loss: 0.0051 - acc: 0.9993
Epoch 1470/1500
 - 0s - loss: 0.0051 - acc: 0.9986
Epoch 1471/1500
 - 0s - loss: 0.0033 - acc: 0.9993
Epoch 1472/1500
 - 0s - loss: 0.0015 - acc: 1.0000
Epoch 1473/1500
 - 0s - loss: 0.0108 - acc: 0.9972
Epoch 1474/1500
 - 0s - loss: 0.0051 - acc: 0.9972
Epoch 1475/1500
 - 0s - loss: 0.0021 - acc: 0.9993
Epoch 1476/1500
 - 0s - loss: 0.0012 - acc: 1.0000
Epoch 1477/1500
 - 0s - loss: 0.0055 - acc: 0.9986
Epoch 1478/1500
 - 0s - loss: 0.0017 - acc: 0.9993
Epoch 1479/1500
 - 0s - loss: 0.0030 - acc: 0.9979
Epoch 1480/1500
 - 0s - loss: 8.0486e-04 - acc: 1.0000
Epoch 1481/1500
 - 0s - loss: 3.7756e-04 - acc: 1.0000
Epoch 1482/1500
 - 